In [4]:
import sys
!{sys.executable} -m pip install psycopg2

  Using cached psycopg2-2.9.5-cp311-cp311-win_amd64.whl (1.2 MB)



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\a764135\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [10]:
!{sys.executable} -m pip install sqlalchemy

  Using cached SQLAlchemy-2.0.7-cp311-cp311-win_amd64.whl (2.0 MB)
  Using cached greenlet-2.0.2-cp311-cp311-win_amd64.whl (192 kB)



[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\a764135\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [2]:
import random

In [3]:
import psycopg2 as pg
import pandas.io.sql as psql

In [4]:
from sqlalchemy import create_engine, text

In [33]:
# connection = pg.connect("host=localhost dbname=pg-datacenter user=pg-datacenter password=Dua2h,i,JY>,,8)U<)aLIguz") # psycopg is deprecated
# dataframe = psql.read_sql('SELECT * FROM products;', connection)
# dataframe

In [9]:
engine = create_engine('postgresql://pg-datacenter:Dua2h,i,JY>,,8)U<)aLIguz@localhost:5432/pg-datacenter')

In [10]:
products_df = pd.DataFrame(engine.connect().execute(text('select * from products;')))

In [11]:
products_df

,id,manufacturer,name,code,colors,price,tax_rate
0,1,Nike,Sneaker,1234567890,"[Black, White]",99.99,19.00
1,2,Adidas,T-Shirt,2345678901,[Red],19.99,19.00
2,3,Puma,Long pants,3456789012,"[Navy, Blue]",49.99,19.00
3,4,The North Face,Winter jacket,4567890123,"[Black, Grey]",199.99,19.00
4,5,Levis,Short pants,5678901234,"[Light Blue, White]",39.99,19.00
...,...,...,...,...,...,...,...
168,169,Nike,Long pants,9012345678901,[Black],99.99,19.00
169,170,Zara,Dress,123456789012,[Red],69.99,19.00
170,171,Levis,Summer jacket,1234567890123,[Blue],129.99,19.00
171,172,Adidas,Sneaker,2345678901234,"[Black, White]",109.99,19.00


In [12]:
shops_df = pd.DataFrame(engine.connect().execute(text('select * from shops;')))

In [13]:
pos_devices_df = pd.DataFrame(engine.connect().execute(text('select * from pos_devices;')))

In [14]:
employees_df = pd.DataFrame(engine.connect().execute(text('select * from employees;')))

In [15]:
customers_df = pd.DataFrame(engine.connect().execute(text('select * from customers;')))

In [17]:
def generate_data(num_rows):
    data = []
    positions = {}
    receipt_id = 1
    position = 1
    
    for i in range(num_rows):
        product_id = random.choice(list(products_df.id.values))
        current_df_row = products_df[products_df["id"]==product_id]
        size = random.choice(['S', 'M', 'L', 'XL'])
        
        quantity = random.choices(range(1, 6), weights=[70, 20, 5, 3, 2])[0]
        discount = random.choices([0, 3, 5, 8, 10, 15, 20, 25, 30, 50], weights=[70, 5, 10, 2, 5, 1, 1, 4, 1, 1])[0]
        
        if discount > 0:
            discount_reason = random.choice(['Seasonal Sale', 'End of Season Clearance', 'Customer Loyalty', 'Holiday Sale'])
        else:
            discount_reason = 'No discount'
            
        color = random.choice(list(current_df_row.colors.values)[0])
        price = round(float(current_df_row["price"].values[0]) * quantity, 2)
        
        data.append((receipt_id, position, product_id, size, color, quantity, price, discount, discount_reason, 19))
        
        if random.random() < 0.3:
            receipt_id += 1
            position = 1
        else:
            position+=1
    return data

In [41]:
all_inserts = generate_data(10000)

In [42]:
lists = []
for ins in all_inserts:
    lists.append(list(ins))

In [43]:
insert_statement = "INSERT INTO public.receipt_positions (receipt_id, position, product_id, size, color, quantity, price, discount, discount_reason, tax_rate) VALUES \n "

In [44]:
for insert_row in all_inserts:
    #print(str(insert_row))
    insert_statement+=str(insert_row)+","
    insert_statement+= "\n "

In [55]:
#print(insert_statement)

In [46]:
cols = ["receipt_id", "position", "product_id", "size", "color", "quantity", "price", "discount", "discount_reason", "tax_rate"]

In [47]:
receipt_positions = pd.DataFrame(lists, columns = cols)

In [48]:
receipt_positions

,receipt_id,position,product_id,size,color,quantity,price,discount,discount_reason,tax_rate
0,1,1,20,S,Red,1,15.99,0,No discount,19
1,2,1,34,M,Navy,1,69.99,0,No discount,19
2,2,2,61,M,Red,2,139.98,0,No discount,19
3,2,3,51,XL,Navy Blue,1,49.99,0,No discount,19
4,2,4,139,S,Blue,1,169.99,0,No discount,19
...,...,...,...,...,...,...,...,...,...,...
9995,3006,1,5,L,Light Blue,1,39.99,0,No discount,19
9996,3006,2,140,XL,White,1,99.99,0,No discount,19
9997,3006,3,110,XL,Green,1,149.99,0,No discount,19
9998,3006,4,173,M,White,1,249.99,25,End of Season Clearance,19


In [50]:
import random
import datetime

all_receipt_inserts = []

start_date = datetime.datetime(2023, 1, 1, 9)
end_date = datetime.datetime(2023, 3, 31, 18)

# Define weights for different days and hours
day_weights = {
    0: 10, # Monday
    1: 10, # Tuesday
    2: 10, # Wednesday
    3: 10, # Thursday
    4: 20, # Friday
    5: 35, # Saturday
}

hour_weights_week = {
    11: 1,   # 11am
    12: 2,   # 12pm
    13: 1,   # 1pm
    14: 2,   # 2pm
    15: 3,   # 3pm
    16: 3, # 4pm
    17: 2, # 5pm
}

hour_weights_sat = {
    9: 1, #9am
    10: 1, #10am
    11: 1,   # 11am
    12: 3,   # 12pm
    13: 4,   # 1pm
    14: 3,   # 2pm
    15: 3,   # 3pm
    16: 2, # 4pm
    17: 2, # 5pm
}

# Define the number of rows to generate
num_rows = max(list(receipt_positions.receipt_id.values))+1

# Define the range of values for each column
shop_ids = list(shops_df["id"].values)
pos_device_ids = list(pos_devices_df["id"].values)
customer_ids = list(customers_df["id"].values)
employee_ids = list(employees_df["id"].values)
delivery_shop_ids = [None, 1, 2, 3]

#                      [11,12,13,14,15,16,17
weekday_hour_weights = [1, 2, 1, 2, 3, 3, 2]
#                       [9,10,11,12,13,14,15,16,17]
weekend_hour_weights = [1, 1, 1, 3, 4, 3, 3, 2, 2]

def generate_timestamp():
    
    weekday = random.choices(range(6), weights=day_weights.values())[0]

    # generate a random date between the start and end dates
    random_date = start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))

    # if the random date is not the correct weekday, adjust it accordingly
    while random_date.weekday() != weekday:
        random_date += datetime.timedelta(days=1)
        
    random_date_splitted = str(random_date).split(" ")[0].split("-")
    year = int(random_date_splitted[0])
    month = int(random_date_splitted[1])
    day = int(random_date_splitted[2])

    if weekday == 5: # Saturday
        hour = random.choices(
            population=list(hour_weights_sat.keys()),
            weights=list(hour_weights_sat.values())
        )[0]
    else:
        hour = random.choices(
            population=list(hour_weights_week.keys()),
            weights=list(hour_weights_week.values())
        )[0]

    # Choose random minute and second
    minute = random.randint(0, 59)
    second = random.randint(0, 59)
        
    return datetime.datetime(year, month, day, hour, minute, second)

rows = []
for i in range(1,num_rows):
    shop_id = random.choice(shop_ids)
    pos_device_id = random.choice(pos_device_ids)
    customer_id = random.choice(customer_ids)
    employee_id = random.choice(employee_ids)
    delivery_shop_id = random.choices(delivery_shop_ids, weights=[90, 5, 2.5, 2.5])[0] # 90% chance of 0, 5% chance of 1, 5% chance of 2 or 3
    
    created_at = generate_timestamp()
    
    current_df = receipt_positions[receipt_positions["receipt_id"]==i]
    
    amount_total = 0
    discount_total = 0
    tax_total = 0
    
    for index, row in current_df.iterrows():
        curr_price = row["price"]
        curr_discount = row["discount"]
        
        if curr_discount > 0:
            factor = curr_discount/100
            reduction = 1-factor
            
            discounted_price = curr_price * reduction
            given_discount = curr_price - discounted_price
        else:
            discounted_price = curr_price
            given_discount = 0
            
        current_tax = discounted_price * (row["tax_rate"] / 100)
        
        amount_total += discounted_price
        discount_total += given_discount
        tax_total += current_tax    
    
    row = (i, shop_id, pos_device_id, customer_id, created_at.strftime("%Y-%m-%d %H:%M:%S"), round(discount_total,2), round(tax_total,2), round(amount_total,2), employee_id, delivery_shop_id)
    rows.append(row)

# Print the rows
for row in rows:
    all_receipt_inserts.append(row)

## create insert string

In [53]:
insert_statement_receipts = "INSERT INTO public.receipts (id, shop_id, pos_device_id, customer_id, created_at, discount_total, tax_total, amount_total, employee_id, delivery_shop_id) VALUES \n "

for insert_row_receipts in all_receipt_inserts:
    #print(str(insert_row))
    insert_statement_receipts+=str(insert_row_receipts)+","
    insert_statement_receipts+= "\n "
    
insert_statement_receipts = insert_statement_receipts.replace("None","NULL")

#print(insert_statement_receipts)

## create df

In [39]:
lists_receipts = []
for ins in all_receipt_inserts:
    lists_receipts.append(list(ins))
    
cols = ["id", "shop_id", "pos_device_id", "customer_id", "created_at", "discount_total", "tax_total", "amount_total", "employee_id", "delivery_shop_id"]

receipts = pd.DataFrame(lists_receipts, columns = cols)
receipts

,id,shop_id,pos_device_id,customer_id,created_at,discount_total,tax_total,amount_total,employee_id,delivery_shop_id
0,1,1,3,66,2023-01-13 13:58:37,0.0,7.60,39.99,29,0
1,2,1,2,132,2023-01-12 12:28:05,2.1,58.49,307.86,21,3
2,3,2,1,237,2023-03-18 17:23:30,0.0,26.60,139.98,25,0
3,4,3,3,277,2023-02-04 12:21:06,8.3,29.77,156.67,27,0
4,5,3,3,248,2023-03-24 11:03:02,0.0,79.79,419.94,19,0
...,...,...,...,...,...,...,...,...,...,...
1528,1529,3,3,244,2023-02-18 13:08:22,0.0,26.60,139.98,28,0
1529,1530,2,3,180,2023-03-28 12:16:12,0.0,117.79,619.96,7,0
1530,1531,3,1,267,2023-02-09 17:07:19,0.0,55.10,289.98,7,0
1531,1532,3,2,74,2023-01-21 14:26:14,25.0,31.34,164.96,9,0


# Create Insert String with ChatGPT input

In [59]:
# INSERT INTO public.receipt_positions (receipt_id, position, product_id, size, color, quantity, price, discount, discount_reason, tax_rate)
# VALUES 
rows = """(1, 1, 10, 'M', 'NO_COLOR', 2, 'NO_PRICE', 0, NULL, 19),(1, 2, 45, 'L', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(2, 1, 88, 'S', 'NO_COLOR', 1, 'NO_PRICE', 5, 'Seasonal Sale', 19),(3, 1, 32, 'XL', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(3, 2, 10, 'M', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(3, 3, 55, 'S', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(4, 1, 20, 'L', 'NO_COLOR', 3, 'NO_PRICE', 0, NULL, 19),(4, 2, 50, 'M', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(4, 3, 75, 'S', 'NO_COLOR', 2, 'NO_PRICE', 0, NULL, 19),(4, 4, 90, 'XL', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(5, 1, 60, 'M', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(5, 2, 80, 'L', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(6, 1, 15, 'S', 'NO_COLOR', 1, 'NO_PRICE', 10, 'Customer Loyalty', 19),(7, 1, 95, 'XL', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(8, 1, 65, 'M', 'NO_COLOR', 2, 'NO_PRICE', 0, NULL, 19),(8, 2, 40, 'L', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(9, 1, 5, 'S', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(10, 1, 70, 'L', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(11, 1, 25, 'M', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19),(11, 2, 35, 'S', 'NO_COLOR', 1, 'NO_PRICE', 0, NULL, 19);"""

In [125]:
insert_statement = "INSERT INTO public.receipt_positions (receipt_id, position, product_id, size, color, quantity, price, discount, discount_reason, tax_rate) VALUES "

In [126]:
last_lower = 0
for i in range(1,21):
    current_row = rows.split(",")[last_lower:i*10]
    current_df_row = products_df[products_df["id"]==int(current_row[2].replace(" ",""))]
    current_price = current_df_row["price"].values[0]
    current_colors = current_df_row["colors"].values[0]
    random_color = random.choice(current_colors)
    
    insert_string = ""
    insert_string = f"{current_row[0]},{current_row[1]},{current_row[2]},{current_row[3]},{random_color},{current_row[5]},{int(current_row[5])*current_price},{current_row[7]},{current_row[8]},{current_row[9]},\n"
    
    insert_statement = insert_statement + insert_string
    
    last_lower = i*10

In [127]:
insert_statement = insert_statement[:-1]

In [128]:
insert_statement

"INSERT INTO public.receipt_positions (receipt_id, position, product_id, size, color, quantity, price, discount, discount_reason, tax_rate) VALUES (1, 1, 10, 'M',Red, 2,299.98, 0, NULL, 19),\n(1, 2, 45, 'L',White, 1,89.99, 0, NULL, 19),\n(2, 1, 88, 'S',White, 1,99.99, 5, 'Seasonal Sale', 19),\n(3, 1, 32, 'XL',Black, 1,39.99, 0, NULL, 19),\n(3, 2, 10, 'M',White, 1,149.99, 0, NULL, 19),\n(3, 3, 55, 'S',Red, 1,149.99, 0, NULL, 19),\n(4, 1, 20, 'L',Red, 3,47.97, 0, NULL, 19),\n(4, 2, 50, 'M',Red, 1,19.99, 0, NULL, 19),\n(4, 3, 75, 'S',White, 2,79.98, 0, NULL, 19),\n(4, 4, 90, 'XL',Green, 1,69.99, 0, NULL, 19),\n(5, 1, 60, 'M',White, 1,189.99, 0, NULL, 19),\n(5, 2, 80, 'L',Black, 1,79.99, 0, NULL, 19),\n(6, 1, 15, 'S',Blue, 1,229.99, 10, 'Customer Loyalty', 19),\n(7, 1, 95, 'XL',Black, 1,119.99, 0, NULL, 19),\n(8, 1, 65, 'M',White, 2,49.98, 0, NULL, 19),\n(8, 2, 40, 'L',Red, 1,119.99, 0, NULL, 19),\n(9, 1, 5, 'S',Light Blue, 1,39.99, 0, NULL, 19),\n(10, 1, 70, 'L',Black, 1,99.99, 0, NULL, 1